In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

In [2]:
#set path
path = '/media/onno/Volume/GFS_T850/'

Calculate Mean Absolute Error for every single vent on lead days 1, 3, 5, 7 and 9

In [30]:
coordinatez = [
(62,54,0,8),#Germany
(54,46,0,8), #Spain
(46,38,0,8), #Ukraine
(62,54,8,16),#Germany
(54,46,8,16), #Spain
(46,38,8,16), #Ukraine
(62,54,16,24),#Germanycw_lon_0_8_lat_62_54.txt
(54,46,16,24), #Spain
(46,38,16,24)] #Ukraine
#loading GFS and ERA5 data for specific box 




#Choose Forecast model: GFS or ERA5RF
fcst_modelz = ['ERA5RF']
for fcst_model in fcst_modelz:
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        file_fcst = file_dic['T850_grid'][fcst_model].format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_rean = file_dic['T850_grid']['ERA5'].format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_clim_p90 = 'era51_mars_t850_79-19_24hourly_90p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_clim_p50 = 'era51_mars_t850_79-19_24hourly_50p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_clim_p10 = 'era51_mars_t850_79-19_24hourly_10p_lon_{}_{}_lat_{}_{}_SMOOTHED_ALL.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        fcst = xr.open_dataset(path+file_fcst,decode_times=False).squeeze()
        #set fcst time data to datetime object
        init_time_fcst = pd.Timestamp('1979-01-01')
        fcst['time'] = [pd.Timedelta(i-fcst.time.values[0],'hours')+init_time_fcst for i in fcst.time.values]
        fcst['lead'] = fcst['lead']//24
        #group forecast by lead time and take daily means
        fcst = fcst.sel(lead=slice(0,9)).groupby('lead').mean()
        rean = xr.open_dataset(path+file_rean).squeeze()
        if int(rean.time[0].dt.hour)!=0:
            rean = rean.assign_coords({'time':rean.time.values - pd.Timedelta(int(rean.time[0].dt.hour),'h')})
        #load percentile climatology. For convenience I set the data to all days in the leap year 2016. That has no further effect on the data
        clim_p90 = xr.open_dataset(path+file_clim_p90).squeeze()
        clim_p90['time']=pd.date_range('2016-01-01',"2016-12-31")
        clim_p10 = xr.open_dataset(path+file_clim_p10).squeeze()
        clim_p10['time']=pd.date_range('2016-01-01',"2016-12-31")
        #load all temperature extremes
        file_pers_hw = 'dates/pw_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
        file_pers_cw = 'dates/pc_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
        file_pers_avg = 'dates/avg_p40_p60_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1)
        pers_hw = np.load(path+file_pers_hw,allow_pickle=True)
        pers_cw = np.load(path+file_pers_cw,allow_pickle=True)
        pers_avg = np.load(path+file_pers_avg,allow_pickle=True)
        temp_extremez = [pers_hw,pers_cw,pers_avg]
        #set column names for csv file
        extremez = ['pw','pc','avg']
        #create index for csv file
        index = pd.date_range(pd.Timestamp('1979-01-01'),pd.Timestamp('2019-12-31'))
        #loop over lead days
        lead_dayz = [3,5,7]
        for lead_day in lead_dayz:
            #create empty pandas dataframe
            #Loop over all extreme events
            for j,temp_extreme in enumerate(temp_extremez):
                #Select persistent or short-lived warm or cold temp extreme
                column = columnz[j]
                df = pd.DataFrame(index=index,columns=np.arange(1,11),dtype=float)
                #Loop over each individual event
                for i,date in enumerate(temp_extreme[:,0]):
                    date = pd.Timestamp(date)
                    begin_date = date - pd.Timedelta(lead_day,'days')
                    #If an event in the first few days of the datset requires a forecast before 1-12-84 skip this event
                    if begin_date<pd.Timestamp('1979-01-01')+pd.Timedelta(lead_day,'days'):
                        continue
                    if np.isin(begin_date,[pd.Timestamp('2014-10-29'),
                                           pd.Timestamp('2014-10-30'),
                                           pd.Timestamp('2014-10-31')]):
                        continue
                    #define date for highest lead day (last day of forecast)
                    end_date = date + pd.Timedelta(9-lead_day,'days')
                    rean_sub = rean.sel(time=slice(begin_date,end_date)).load()
                    fcst_sub = fcst.sel(time=begin_date).load()
                    diff = np.abs(fcst_sub.t - rean_sub.t.values).values
                    df.loc[date]=diff
                df = df.dropna(how='all')
                df.to_csv(path+'anom_files/Mean_Absolute_Error_lon_{}_{}_lat_{}_{}_lead_day_{}_{}.csv'.format(lon_0,lon_1,
                                                                                           lat_0,lat_1,lead_day,extremez[j]))
